<a href="https://colab.research.google.com/github/TottiPuc/clasificador/blob/master/pytorch_model_to_tensorflow_with_ONNX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.optim as optim
import pandas as pd
import numpy as np

In [2]:
!wget https://github.com/TottiPuc/clasificador/raw/master/storepurchasedata_large.csv

--2021-05-29 03:20:58--  https://github.com/TottiPuc/clasificador/raw/master/storepurchasedata_large.csv
Resolving github.com (github.com)... 13.114.40.48
Connecting to github.com (github.com)|13.114.40.48|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/TottiPuc/clasificador/master/storepurchasedata_large.csv [following]
--2021-05-29 03:20:58--  https://raw.githubusercontent.com/TottiPuc/clasificador/master/storepurchasedata_large.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17114 (17K) [text/plain]
Saving to: ‘storepurchasedata_large.csv’

storepurchasedata_l 100%[===================>]  16.71K  --.-KB/s    in 0.002s  

2021-05-29 03:20:59 (9.26 MB/s) - ‘storepurchasedata_large.csv’ save

In [3]:
df = pd.read_csv('storepurchasedata_large.csv')

In [4]:
df.describe()

,Age,Salary,Purchased
count,1554.000000,1554.000000,1554.000000
mean,44.296010,57042.471042,0.694981
std,17.462458,21209.244800,0.460564
min,18.000000,20000.000000,0.000000
25%,27.000000,46000.000000,0.000000
50%,43.000000,60000.000000,1.000000
75%,62.000000,66000.000000,1.000000
max,69.000000,96000.000000,1.000000


In [5]:
X = df.iloc[:,:-1].values
Y = df.iloc[:,-1].values

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.20, random_state=0)

In [9]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [10]:
Xtrain_ = torch.from_numpy(X_train).float()
Xtest_ = torch.from_numpy(X_test).float()

In [11]:
Xtrain_.shape, Xtest_.shape

(torch.Size([1243, 2]), torch.Size([311, 2]))

In [39]:
Xtrain_.dtype

torch.float32

In [12]:
Ytrain_ = torch.from_numpy(y_train)
Ytest_ = torch.from_numpy(y_test)

In [13]:
input_size=2
output_size=2
hidden_size=10

In [14]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = torch.nn.Linear(input_size,hidden_size)
    self.fc2 = torch.nn.Linear(hidden_size,hidden_size)
    self.fc3 = torch.nn.Linear(hidden_size,output_size)

  def forward(self,X):
    X=torch.relu((self.fc1(X)))
    X=torch.relu((self.fc2(X)))
    X=self.fc3(X)

    return F.log_softmax(X,dim=1)

In [15]:
model=Net()

In [16]:
optimizer = optim.Adam(model.parameters(),lr=0.01)
loss_fn = nn.NLLLoss()

In [17]:
epochs=100

In [18]:
for epoch in range(epochs):
  optimizer.zero_grad()
  Ypred = model(Xtrain_)
  loss = loss_fn(Ypred, Ytrain_)
  loss.backward()
  optimizer.step()
  print('Epoch ', epoch, 'loss ', loss.item())

Epoch  0 loss  0.6560925245285034
Epoch  1 loss  0.6358411908149719
Epoch  2 loss  0.618733286857605
Epoch  3 loss  0.6045369505882263
Epoch  4 loss  0.5918965935707092
Epoch  5 loss  0.5792876482009888
Epoch  6 loss  0.5661488175392151
Epoch  7 loss  0.552024781703949
Epoch  8 loss  0.5389926433563232
Epoch  9 loss  0.5263939499855042
Epoch  10 loss  0.5135021805763245
Epoch  11 loss  0.5002800226211548
Epoch  12 loss  0.4861507713794708
Epoch  13 loss  0.4709118604660034
Epoch  14 loss  0.45491254329681396
Epoch  15 loss  0.4377923011779785
Epoch  16 loss  0.41990527510643005
Epoch  17 loss  0.40137672424316406
Epoch  18 loss  0.38238754868507385
Epoch  19 loss  0.363183856010437
Epoch  20 loss  0.3439658284187317
Epoch  21 loss  0.32534855604171753
Epoch  22 loss  0.30754464864730835
Epoch  23 loss  0.2906007170677185
Epoch  24 loss  0.27448606491088867
Epoch  25 loss  0.25907906889915466
Epoch  26 loss  0.24408462643623352
Epoch  27 loss  0.22944794595241547
Epoch  28 loss  0.21569

In [19]:
torch.from_numpy(sc.transform(np.array([[40,22000]]))).float()

tensor([[-0.1704, -1.5960]])

In [20]:
y_cust_20_40000 = model(torch.from_numpy(sc.transform(np.array([[40,22000]]))).float())
y_cust_20_40000

tensor([[-0.0158, -4.1538]], grad_fn=<LogSoftmaxBackward>)

In [21]:
_,predicted_20_40000 = torch.max(y_cust_20_40000,-1)
predicted_20_40000

tensor([0])

# Export model to ONNX format

In [22]:
sample_tensor = torch.from_numpy(sc.transform(np.array([[40,20000]]))).float()

In [23]:
torch.onnx.export(model,sample_tensor,'customer.onnx',export_params=True)

In [24]:
!ls

customer.onnx  sample_data  storepurchasedata_large.csv


##load model to tensorflow

In [25]:
!pip install onnx

     |████████████████████████████████| 12.2MB 290kB/s 


In [26]:
!pip install onnx_tf

     |████████████████████████████████| 225kB 3.0MB/s 
     |████████████████████████████████| 686kB 14.9MB/s 


In [28]:
import onnx
import onnx_tf
import tensorflow_addons
from onnx_tf.backend import prepare

In [29]:
onnx_model=onnx.load('customer.onnx')

In [30]:
tf_rep = prepare(onnx_model)

In [40]:
prediction = tf_rep.run((sc.transform(np.array([[42,50000]],dtype=np.float32))))

In [41]:
prediction

Outputs(_0=array([[-1.4355416 , -0.27179092]], dtype=float32))